<a href="https://colab.research.google.com/github/Antonioarf/blox/blob/master/recomenda%C3%A7%C3%A3o/Recommender-t2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAPIENS 1

In [0]:
# !pip install -U -q PyDrive
import heapq
import random
import pandas as pd;
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
from google.colab import drive
import torch
from joblib import dump, load
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# import pickle
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression

In [0]:
df1 = pd.read_csv("https://github.com/Antonioarf/blox/blob/master/recomenda%C3%A7%C3%A3o/df_1_spaiens.csv?raw=true")
df2 = pd.read_csv("https://raw.githubusercontent.com/Antonioarf/blox/master/recomenda%C3%A7%C3%A3o/df_2_sapiens.csv") 
df5 = df1

## limpesa e dummify do df1
### estamos trabalhando com 113 blox e 678 alunos

In [0]:
def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=None)], axis=1)
    return df2

def deletar(coluna):
    lista = list (coluna.unique()) #.tolinst()  
    for e in lista:
        coluna = coluna.replace(e,lista.index(e))
    return coluna

In [0]:
df1.head()
dfx = df1
dfx = dfx.set_index('nome_do_aluno')
dfx = dfx.sort_values("nome_do_aluno")
entao = dfx['nome_blox']

In [0]:
df1 = dummify(df1, ['nome_blox'])
lista = df1.columns.tolist()[5:]

In [0]:
aggregation_functions = {}
aggregation_functions['course_name'] = 'first'
for e in lista:
    aggregation_functions[e] = 'sum'
df1 = df1.groupby(df1['nome_do_aluno']).aggregate(aggregation_functions)

In [0]:
lista = ["course_name"]
for e in df1.columns[1:]:
    lista.append(e[10:])
df1.columns = lista

## Limpesa e tratamento do df2

## Primeira tentativa 

In [0]:
df_t = pd.DataFrame()
for e in df1.columns[1:]:
    lista = list (df2.loc[df2['nome_blox'] == e ]["area_funcional"])
    df_t[e] = df1[e].replace(1, lista[0] )#.replace(0,np.nan)

In [0]:
df3 = df1
df3 = df3.drop(list(df1.columns),axis=1)
lista = []
for e in df_t.values.tolist():
    lista.append(list(filter(lambda a: a != 0, e)))
df3['lista'] = lista

In [0]:
lll = []
for i in df3['lista']:
    k = i[0]
    lista = []
    for e in df2["area_funcional"].unique():
        lista.append(e.count(k))
    lll.append(lista)
df4 = pd.DataFrame.from_records(lll,columns = df2["area_funcional"].unique())

In [0]:
df4['corse_name'] = df1['course_name'].tolist()
df4['corse_name'] = df4["corse_name"].astype('category')
lista=[]
for e in dfx.index.unique():
    lista.append(dfx.loc[dfx.index == e]["nome_blox"].tolist())
df4["resp"] = lista

In [38]:
df4.index = df3.index
# dfx = dfx.set_index('nome_do_aluno')
dfx = dfx.drop(["nome_blox","course_name","modalidade","ch","nota"],axis = 1)
dfx.cloumns = df4.columns
dfx.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


(33300, 0)

In [0]:
dfx = dfx.merge(df4, left_on= 'nome_do_aluno', right_on= 'nome_do_aluno')

In [40]:
dfx = dummify(dfx, 'corse_name')
dfx['resp']= entao.astype('category')
dfx.shape

(33300, 34)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(dfx ,dfx["resp"], test_size=0.1,random_state=12)
lis = y_train.unique()
X_train = X_train.drop(["resp"],axis =1)
X_test = X_test.drop(["resp"],axis =1)

## Salvando o modelo

In [0]:
def regr(x,y):
    rdm = OneVsRestClassifier(SVC(kernel='linear'))
    # rdm = SVC(kernel='linear')
    rdm = rdm.fit(x,y)
    # j = rdm.decision_function(X_t)
    return rdm

In [0]:
svc = regr(X_train,y_train)

In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [45]:
!ls /content/gdrive/'My Drive'

 Book1.xlsx
'Colab Notebooks'
'Cópia de custom-query-offering-production-fiep'
'Cópia de NOVO CINE de ESPM - Pós - Matriz de Referência.gsheet'
 ementas.xlsx
'ESPM - GRADE 1.gsheet'
'ESPM - GRADE 1.xlsx'
 eu.gsheet
'Matrizes em Excel.gsheet'
'Matrizes em Excel.xlsx'
 ml-blox.json
 recomender.pt
'Relatório 1'
'Relatório sem título'
'Relatório sem título (1)'
'Relatório sem título (2)'
'Relatório sem título (3)'
'Relatório sem título (4)'
'Relatório sem título (5)'
'Relatório sem título (6)'


In [0]:
model_save_name = 'recomender.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(svc, path)

## Reabrindo o modelo

In [0]:
model_save_name = 'recomender.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
teste_de_nome = torch.load(path)

In [48]:
teste_de_nome 

OneVsRestClassifier(estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma='scale', kernel='linear', max_iter=-1,
                                  probability=False, random_state=None,
                                  shrinking=True, tol=0.001, verbose=False),
                    n_jobs=None)

## Contrução da recomendação

In [0]:
def sugestao(rdm,X):
    j = rdm.decision_function(X)
    lista = list (X.index)
    sugestoes = []
    for e, t in zip (j, range(len (j)-1)):
        sugestoes.append( list(set(list([lis[n] for n in heapq.nlargest(5, range(len(e)), e.take)])) - set(list(df5.loc[df5['nome_do_aluno'] == lista[t]]["nome_blox"]))))
    return sugestoes

In [0]:
deu = sugestao(teste_de_nome,X_test)

In [51]:
lista = list (X_test.index)
x =  random.randint(0,len(lista))
print(x)
print (list(df5.loc[df5['nome_do_aluno'] == lista[x]]["nome_blox"]))
print ("\n")
for e in deu[x]:
  print (e)

1991
['Rondo.Agro', 'Economia II', 'PROFISSIONALIZE SUA EMPRESA', 'Mercado de capitais: Um passo para aplicar.', 'LOGÍSTICA DE A a Z', 'Inglês', 'Administrando as Finanças da Organização ', 'Agro.export', 'Bye Bye Estoques', 'Ticket de Negócios', 'Faça amor, não faça guerra', 'On ou off não importa! A comunicação tem que ser Integrada.', 'Criando valor para a Internacionalização', 'Estratégia de Sísifo', 'Cadeia de Suprimentos fora da Caixinha', 'O Despertar', 'Aprovado em 8 Concursos Públicos: o que ele fez?', 'Internacionalizando negócios', 'Consultoria - Eleve o potencial competitivo das empresas', 'SUPPLY CHAIN', 'REDAÇÃO NOTA MIL- Saiba como escrever e se dar bem.', 'Exportar é o que importa!', 'LEALDADE: O PARCEIRO CERTO PARA SEU NEGÓCIO', 'TRANSFORMANDO OS PIXELS EM RESULTADO', 'De Rondônia para o Mundo', 'NEGÓCIOS FAMILIARES', 'QUER TER UMA CARREIRA DE SUCESSO?', 'SERÁ QUE SEI PESQUISAR? VAMOS APRENDER!', 'CONHECIMENTO SEM FRONTEIRA', 'CONTADOR RAIZ X CONTADOR NUTELLA', 'Guarda

In [0]:
# globals() 0,3 == 9990